In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from collections import Counter
from fuzzywuzzy import process, fuzz
from tqdm import tqdm
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../")

import src.biblioteca as bb
sys.path.append("../")
import src.soportecleaning as spcl

Note to self:
en este archivo van cosas que luego van a ser tres archivos distintos.
- Limpieza
- EDA
- Charts
- 
Recomendable separarlo y trabajarlo de forma independiente

# Input files

Base data file

In [2]:
with open('../data/pickle/dataset_usa_base.pickle', 'rb') as base_data:
    base_data = pickle.load(base_data)
print(base_data.shape)
base_data.head(1)

(455057, 9)


,title,rank,date,artist,url,region,chart,trend,streams
8246,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000


LastFM extraction

In [3]:
with open('../data/pickle/spotify_dataset_lastfm.pickle', 'rb') as data_lastfm:
    data_lastfm = pickle.load(data_lastfm)
data_lastfm.drop_duplicates(inplace=True)
print(data_lastfm.drop_duplicates().shape)
data_lastfm.head(1)

(14659, 17)


,url,artist,track,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap


Spotify extraction

In [4]:
with open('../data/pickle/spotify_dataset_audioFeatures.pickle', 'rb') as data_spotify:
    data_spotify = pickle.load(data_spotify)
data_spotify.drop_duplicates(inplace=True)
print(data_spotify.drop_duplicates().shape)
data_spotify.head(1)

(13989, 21)


,url,artist,track,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Merging all data

In [5]:
df = base_data.merge(data_lastfm, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          456306
left_only          0
right_only         0
dtype: int64

In [6]:
df.drop(['_merge'], axis = 1, inplace = True)
df = df.merge(data_spotify, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          442906
left_only      13924
right_only         0
dtype: int64

In [7]:
df.drop(['artist_y', 'track_x', 'artist', 'track_y', '_merge'], axis = 1, inplace= True)
df.rename({'artist_x':'artist'}, inplace=True, axis=1)
print(df.shape)
df.head(1)

(456830, 41)


,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Cleaning data

In [8]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


Highest rank in list + Weeks in charts

In [9]:
lista_url = df['url'].unique().tolist()
len(lista_url)

14650

In [10]:
ranks = {
    'url':[],
    'highest_rank':[],
    'weeks_in_charts':[]
}

In [11]:
def highest_rank (url, row):
    return row[row['url'] == url].sort_values(by = 'rank').iloc[0,1]

In [12]:
def weeks_in_charts (url):
    return df[df['url'] == url].shape[0]

In [13]:
for url in tqdm(lista_url):
    ranks['url'].append(url)
    ranks['highest_rank'].append(df[df['url'] == url].sort_values(by = 'rank').iloc[0,1])
    ranks['weeks_in_charts'].append(df[df['url'] == url].shape[0])

100%|██████████| 14650/14650 [13:09<00:00, 18.56it/s]


In [14]:
print(len(ranks['url']), len(ranks['highest_rank']), len(ranks['weeks_in_charts']))

14650 14650 14650


In [15]:
df = df.merge(pd.DataFrame(ranks), on = 'url', indicator= True)
df.value_counts('_merge')

_merge
both          456830
left_only          0
right_only         0
dtype: int64

In [16]:
df.drop(['_merge'], axis = 1, inplace=True)

Main music genre

Primo los géneros de las canciones -- si no hubiera datos, cojo los valores del artista

In [17]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429


In [18]:
df['music_genre'] = np.where(df['aritist_genre_0'].isnull() == True, df['track_genre_0'].str.lower(), df['aritist_genre_0'].str.lower())
df['music_genre'].value_counts()

hip-hop         131641
pop              65166
rap              30438
country          25113
rnb              22598
                 ...  
wave                 1
calypso              1
southern rap         1
salsa                1
c                    1
Name: music_genre, Length: 363, dtype: int64

In [19]:
cuenta_generos = Counter(genres for genres in df['music_genre'])
print(f"Hay {len(cuenta_generos)} generos diferentes.")

Hay 2976 generos diferentes.


In [20]:
dict_genres = dict(cuenta_generos.most_common(24))

In [21]:
fuzz.ratio('hip-hop', "hip hop")

86

In [22]:
def music_genres(column, genres):
    max = 0
    for key in genres.keys():
        try:
            similar = fuzz.ratio(column, key)
            if similar > max:
                max = similar
                genre = key     
        except:
            return "other"
    if max > 60:
        return genre
    else:
        return "other"

In [23]:
df["clean_music_genre"] = df.apply(lambda x: music_genres(x["music_genre"], dict_genres), axis = 1)
df["clean_music_genre"].value_counts()

hip-hop             132882
other                91631
pop                  65166
rap                  30446
country              25295
rnb                  22649
trap                 19979
electronic           11784
indie                11121
rock                  6456
soul                  5459
acoustic              3902
indie pop             3526
classic rock          3410
jazz                  3110
k-pop                 2783
house                 2610
lo-fi                 2416
indie rock            2098
latin                 2065
female vocalists      1823
black metal           1772
cloud rap             1600
reggaeton             1443
seen live             1404
Name: clean_music_genre, dtype: int64

Gender from artists

ojo, hay un poco de error -- volver a pasarlo, pero solo con el nombre de los artistas (que no sean grupos)

In [24]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
121377,Selfish,61,2020-04-26,Madison Beer,https://open.spotify.com/track/4PV0uE5pZSh44E3...,United States,top200,MOVE_UP,318662.000,"15 Feb 2020, 23:18","10 May 2014, 13:35","Madison Elle Beer (born March 5, 1999 in Jeric...","Madison Elle Beer (born March 5, 1999 in Jeric...",pop,female vocalists,rnb,indie,indie pop,pop,sad,female vocalists,rnb,adult contemporary,spotify:track:5UY8jf31X2hJkiAualFTyh,0.375,0.461,9.000,-6.202,1.000,0.028,0.627,0.000,0.386,0.233,75.217,audio_features,5UY8jf31X2hJkiAualFTyh,https://api.spotify.com/v1/tracks/5UY8jf31X2hJ...,https://api.spotify.com/v1/audio-analysis/5UY8...,223270.000,4.000,28,134,pop,pop
261162,Shotta Flow,45,2019-06-16,NLE Choppa,https://open.spotify.com/track/4dAMdQ6g4kGmnc1...,United States,top200,MOVE_UP,391647.000,NaN,"25 Mar 2019, 13:04",Memphis teen rap sensation NLE Choppa has seen...,Memphis teen rap sensation NLE Choppa has seen...,trap,Hip-Hop,rap,memphis,hip hop,NaN,NaN,NaN,NaN,NaN,spotify:track:5BJMeoCXXgbRAWfp6fTulr,0.894,0.511,2.000,-4.768,1.000,0.421,0.025,0.000,0.140,0.567,120.077,audio_features,5BJMeoCXXgbRAWfp6fTulr,https://api.spotify.com/v1/tracks/5BJMeoCXXgbR...,https://api.spotify.com/v1/audio-analysis/5BJM...,176632.000,4.000,2,282,trap,trap
212151,idontwannabeyouanymore,144,2018-10-27,Billie Eilish,https://open.spotify.com/track/41zXlQxzTi6cGAj...,United States,top200,MOVE_DOWN,247646.000,"22 Jan 2022, 07:44","15 Aug 2016, 17:51",Billie Eilish Pirate Baird O'Connell (born Dec...,Billie Eilish Pirate Baird O'Connell (born Dec...,pop,indie pop,female vocalists,electropop,indie,indie pop,indie,pop,2010s,Billie Eilish,spotify:track:41zXlQxzTi6cGAjpOXyLYH,0.483,0.412,7.000,-8.461,1.000,0.040,0.737,0.000,0.116,0.247,170.163,audio_features,41zXlQxzTi6cGAjpOXyLYH,https://api.spotify.com/v1/tracks/41zXlQxzTi6c...,https://api.spotify.com/v1/audio-analysis/41zX...,203569.000,3.000,45,137,pop,pop
32632,Ric Flair Drip (& Metro Boomin),21,2018-05-21,Offset,https://open.spotify.com/track/7sO5G9EABYOXQKN...,United States,top200,MOVE_DOWN,709736.000,"06 May 2021, 23:54","10 Nov 2007, 03:21","Kiari Kendrell Cephus (born December 14, 1991)...","Kiari Kendrell Cephus (born December 14, 1991)...",trap,Hip-Hop,rap,ambient,techno,rap,trap,Hip-Hop,hip hop,Awesome,spotify:track:7sO5G9EABYOXQKNPNiE9NR,0.880,0.428,9.000,-8.280,1.000,0.206,0.149,0.000,0.114,0.333,100.007,audio_features,7sO5G9EABYOXQKNPNiE9NR,https://api.spotify.com/v1/tracks/7sO5G9EABYOX...,https://api.spotify.com/v1/audio-analysis/7sO5...,172800.000,4.000,2,1974,trap,trap
347121,Levitating (feat. DaBaby),6,2021-04-18,Dua Lipa,https://open.spotify.com/track/5nujrmhLynf4yMo...,United States,top200,MOVE_UP,1038214.000,"06 Jul 2021, 13:36","26 Apr 2016, 03:42","Dua Lipa (born August 22, 1995) is an English ...","Dua Lipa (born August 22, 1995) is an English ...",pop,electropop,synthpop,british,female vocalists,pop,Disco,nu-disco,2020,popstars dua lipa with dababy,spotify:track:5nujrmhLynf4yMoMtj8AQF,0.702,0.825,6.000,-3.787,0.000,0.060,0.009,0.000,0.067,0.915,102.977,audio_features,5nujrmhLynf4yMoMtj8AQF,https://api.spotify.com/v1/tracks/5nujrmhLynf4...,https://api.spotify.com/v1/audio-analysis/5nuj...,203064.000,4.000,3,427,pop,pop


In [34]:
## funciona pero tiene un poco de error
def generos(col):
    for key, value in bb.dict_gender.items():
        for k in key:
            for item in value:
                try:
                    if item in col.lower():
                        return key
                except:
                    return np.nan
    return np.nan

In [35]:
def generos_2(col):
    for key, value in bb.dict_gender.items():
        try:
            if value in col.lower():
                return key
        except:
            return np.nan
    return np.nan

In [27]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429,hip-hop,hip-hop


In [38]:
#df["gender"] = df.apply(lambda x: generos(x['content'], bb.dict_gender), axis=1)
df["gender"] = df['summary'].apply(generos)

In [39]:
df['gender'].value_counts() / df.shape[0]

male         0.535
female       0.140
group        0.111
non-binary   0.023
Name: gender, dtype: float64

Age from artist

-- taking the data from the summary

In [48]:
df['birthday'] =  df['content'].apply(lambda x: re.findall(r'\w{1,15} \d{1,2}, \d{4}',str(x))).str[0]
df['birthday'].value_counts()

December 2, 1998     10839
July 4, 1995         10700
January 23, 1998      9272
October 24, 1986      8719
December 18, 2001     7645
                     ...  
March 10, 1984           1
November 20, 1981        1
May 26, 2013             1
March 1, 1927            1
October 6, 2000          1
Name: birthday, Length: 983, dtype: int64

In [33]:
df.sample()

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre,gender,birthday
413323,THOT!,12,2020-01-11,"TOKYO’S REVENGE, ZEDSU",https://open.spotify.com/track/2St8HeG23Nzp4NY...,United States,viral50,SAME_POSITION,NaN,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/TOKYO%E2%8...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spotify:track:2St8HeG23Nzp4NYaWTNtuM,0.946,0.592,8.000,-7.908,1.000,0.230,0.018,0.000,0.460,0.418,100.023,audio_features,2St8HeG23Nzp4NYaWTNtuM,https://api.spotify.com/v1/tracks/2St8HeG23Nzp...,https://api.spotify.com/v1/audio-analysis/2St8...,159709.000,4.000,5,44,NaN,other,NaN,NaN


df.dtypes

In [46]:
df.dtypes

title                 object
rank                   int64
date                  object
artist                object
url                   object
region                object
chart                 object
trend                 object
streams              float64
published_track       object
published             object
summary               object
content               object
aritist_genre_0       object
aritist_genre_1       object
aritist_genre_2       object
aritist_genre_3       object
aritist_genre_4       object
track_genre_0         object
track_genre_1         object
track_genre_2         object
track_genre_3         object
track_genre_4         object
URI                   object
danceability         float64
energy               float64
key                  float64
loudness             float64
mode                 float64
speechiness          float64
acousticness         float64
instrumentalness     float64
liveness             float64
valence              float64
tempo         

In [49]:
df['playlist_date'] = pd.to_datetime(df['date'])
df['birthday_date'] = pd.to_datetime(df['birthday'])

In [50]:
df['birthday_date'].value_counts()

1998-12-02    10839
1995-07-04    10700
1998-01-23     9272
1986-10-24     8719
2001-12-18     7645
              ...  
1984-03-10        1
1981-11-20        1
2013-05-26        1
1927-03-01        1
2000-10-06        1
Name: birthday_date, Length: 983, dtype: int64

In [51]:
from datetime import date

def calculate_age(birthday):
    today = date.today()
    return today.year - birthday.year - ((today.month, today.day) < (birthday.month, birthday.day))

In [52]:
df['age'] = df['birthday_date'].apply(calculate_age)
df['age'].value_counts()

24.000     29035
27.000     24258
30.000     16589
23.000     13114
5.000      12234
           ...  
120.000        5
98.000         3
87.000         2
91.000         2
72.000         1
Name: age, Length: 105, dtype: int64

# EDA

# Questions / hypothesis

Most streamed artists

In [ ]:
top10 = df.groupby(['artist'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10['ratio'] = top10['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10

Most streamed music genres

In [ ]:
top10genre = df.groupby(['aritist_genre_0'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10genre['ratio'] = top10genre['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10genre[top10genre['ratio'] > 0]

Artist evolution

In [ ]:
lista_artistas = list(top10['artist'])
df_top10 = df[df['artist'].isin(lista_artistas)]
df_top10.head()

In [ ]:
num_rows = 20
years = list(range(1990, 1990 + num_rows))
data_preproc = pd.DataFrame({
    'Year': years, 
    'A': np.random.randn(num_rows).cumsum(),
    'B': np.random.randn(num_rows).cumsum(),
    'C': np.random.randn(num_rows).cumsum(),
    'D': np.random.randn(num_rows).cumsum()})

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
test = pd.pivot_table(df_top10[df_top10['chart'] == 'top200'][['date', 'artist', 'rank']], index = 'date', values = ['rank'], columns = 'artist').reset_index().droplevel(level=0, axis =1)
test.rename({'':'date'}, axis = 1, inplace=True)
print(test.shape)
test.head()

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
sns.set(rc={'figure.figsize':(50,8.27)})

c = sns.lineplot(data = test, x = test['date'].unique().tolist(), y = 'Ariana Grande')
c.set_xticklabels(labels = test['date'].unique().tolist(), rotation=90)

Music genre evolution

In [ ]:
df

Genre distibution

How was the music during covid